In [1]:
## import warnings
import time
import math
import os
import glob
from pyhere import here
from datetime import date
import re
from collections import Counter

import numpy as np
import pandas as pd
import geopandas
import pickle

import pyarrow
import itertools
import multiprocessing
import p_tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import train_test_split, KFold, LeaveOneGroupOut, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.metrics import r2_score
from scipy.stats import spearmanr,  pearsonr

In [23]:
climate_df

,year,district,yield_mt,0,1,2,3,4,5,6,7,8,9,10,11
0,2009,Chibombo,-0.090410,0.549876,-0.138716,-0.203109,-0.112067,0.084830,0.231024,0.221465,0.166326,0.252925,0.336969,1.110156,0.805603
1,2009,Kabwe,-0.103302,0.172679,-0.287708,-0.300886,-0.278230,0.025778,0.305693,0.219981,0.153985,0.227204,0.116862,0.635033,1.026011
2,2009,Kapiri Mposhi,-0.090485,0.089158,-0.251909,-0.221435,-0.253983,0.032938,0.205619,0.245590,0.270501,0.178483,0.209953,0.782114,0.661518
3,2009,Mkushi,-0.081334,0.092215,-0.046389,-0.114967,0.005866,0.203737,0.324349,0.266050,0.375827,0.350925,0.294461,0.262816,0.790489
4,2009,Mumbwa,-0.086725,0.372638,-0.215644,-0.357058,-0.372404,-0.197036,0.007611,0.109418,0.198998,0.143413,0.264678,1.367134,0.606310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,2021,Lukulu,0.039854,0.235526,0.107345,0.104234,0.012547,0.016331,-0.027464,-0.006029,0.084417,0.238132,0.081386,-0.037082,0.302118
932,2021,Mongu,0.054079,0.055339,-0.047522,0.102702,-0.038394,-0.079527,-0.083853,-0.070213,0.024666,0.372201,0.086621,-0.128091,0.298678
933,2021,Senanga,0.028426,0.189054,0.088814,0.116410,-0.033478,-0.040581,-0.026247,0.009117,0.062334,0.424772,0.194880,0.009377,0.267773
934,2021,Sesheke,-0.006688,0.344972,0.365315,0.061507,-0.093383,0.031836,0.081841,0.118957,0.049468,0.350476,-0.121629,-0.003233,0.375527


In [26]:
climate_df = pd.read_csv(here('data', 'climate', 'climate_summary.csv'))
climate_df = climate_df.dropna()
drop_cols = ['year', 'district', 'yield_mt']
ndvi_cols = climate_df.columns[climate_df.columns.to_series().str.contains('ndvi')]
keep_cols = [*ndvi_cols, *drop_cols]
# climate_df = climate_df.loc[:, keep_cols]
# climate_df = climate_df[climate_df.year >= 2016]
hot_encode = True
# hot_encode = False
# anom = True
anom = False
if anom:
    hot_encode = False
else:
    pass


crop_yield = climate_df.copy().loc[:, tuple(drop_cols)].reset_index(drop = True)
crop_yield["log_yield"] = np.log10(crop_yield.yield_mt.to_numpy() + 1)

#########################################    HOT ENCODE    ######################################### 
if hot_encode:
    drop_cols.remove('district')
    climate_df = pd.get_dummies(climate_df, columns = ["district"], drop_first = False)
else:
    pass

#########################################    STANDARDIZE FEATURES    #########################################    
climate_df = climate_df.set_index(drop_cols) 
climate_df_scaled = StandardScaler().fit_transform(climate_df.values)
climate_df = pd.DataFrame(climate_df_scaled, index=climate_df.index).reset_index()
climate_df.columns = climate_df.columns.astype(str)

#########################################     CALCULATE ANOMALY   #########################################
if anom:    
    climate_df['yield_mt'] = np.log10(climate_df.yield_mt.to_numpy() + 1)
    climate_df.set_index(['year', 'district'], inplace=True)
    var_cols = climate_df.columns
    climate_df = climate_df[var_cols] - climate_df.groupby(['district'], as_index=True)[var_cols].transform('mean')
    climate_df.reset_index(drop=False, inplace=True)
else:
    pass

#########################################     K-FOLD SPLIT    #########################################
x_all = climate_df.drop(drop_cols, axis = 1) 
if anom:
    y_all = climate_df.yield_mt
else:
    y_all = np.log10(climate_df.yield_mt.to_numpy() + 1)
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=0)

#########################################     K-FOLD CV   ###########################################
### SETUP
alphas = {'alpha': np.logspace(-8, 8, base = 10, num = 17)}
kfold = KFold()
ridge = Ridge()    
### GRID SEARCH - FINDING BEST REGULARIZATION PARAMETER
kfold_ridge_reg = GridSearchCV(ridge, alphas, scoring = 'r2', cv = kfold)
kfold_ridge_reg.fit(x_train, y_train)
kfold_best_model = kfold_ridge_reg.best_estimator_
### PREDICT - PREDICTING WITH BEST HYPERPARAMETER
kfold_val_predictions = cross_val_predict(kfold_best_model, X = x_train, y = y_train, cv = kfold)   
y_pred_train_k = kfold_best_model.predict(x_train)
y_pred_test_k  = kfold_best_model.predict(x_test)
print(f'Val  R2: {r2_score(y_train, kfold_val_predictions):0.2f}\nTest R2: {r2_score(y_test, y_pred_test_k):0.2f}')

Val  R2: 0.75
Test R2: 0.75


In [27]:
climate_df

,year,yield_mt,0,1,2,3,4,5,6,7,...,98,99,100,101,102,103,104,105,106,107
0,2009,1.562466,-0.256140,-1.633278,0.063323,-1.477381,0.010094,-0.609141,-0.352131,-0.570546,...,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678
1,2009,1.899954,-0.200749,-1.274179,0.394552,-1.371458,-0.123867,-0.630522,-0.403289,-0.542623,...,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678
2,2009,1.800877,-0.392781,-0.760635,0.818738,-1.200054,-0.071096,-0.545581,-0.240445,-0.272518,...,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678
3,2009,2.678332,0.032232,-0.886477,0.942658,-1.358270,-0.136040,0.643146,0.570579,0.284906,...,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678
4,2009,1.327876,-0.553432,-1.308566,-0.063918,-1.238393,-0.019673,-0.603616,-0.301584,-0.447448,...,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,2021,1.008066,-0.121453,-0.156174,-0.259421,-0.347897,-0.415585,-0.212967,-0.342159,-0.246668,...,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678
932,2021,0.813282,-0.471733,-0.314685,-0.622264,-0.436788,-0.452527,-0.244472,-0.287769,-0.070076,...,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678
933,2021,0.561860,-1.045936,-0.770841,-0.960080,-0.642546,-0.543265,-0.356600,-0.245047,-0.492904,...,-0.118678,-0.118678,8.426150,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678
934,2021,0.538047,-1.124894,-0.927741,-1.146688,-0.821516,-0.659158,-0.481288,-0.176073,-0.677290,...,-0.118678,-0.118678,-0.118678,-0.118678,8.426150,-0.118678,-0.118678,-0.118678,-0.118678,-0.118678


In [28]:
predictions = pd.concat([x_train, x_test])
predictions['split'] = np.concatenate((np.repeat('train', len(x_train)), np.repeat('test', len(x_test))))
predictions['kfold_cv_predictions'] = np.concatenate((np.maximum(kfold_val_predictions, 0),  np.repeat(np.nan, len(x_test))))
predictions = predictions[['split', 'kfold_cv_predictions']]
predictions = crop_yield.join(predictions)
predictions["district_yield_mean"] = predictions.groupby('district')['log_yield'].transform('mean')
predictions["demean_yield"] = predictions["log_yield"] - predictions["district_yield_mean"]

predictions["prediction"] = np.maximum(kfold_best_model.predict(x_all), 0)
predictions["district_prediction_mean"] = predictions.groupby('district')['prediction'].transform('mean')
predictions["demean_prediction"] = predictions["prediction"] - predictions["district_prediction_mean"]
predictions["demean_cv_prediction"] = predictions["kfold_cv_predictions"] - predictions["district_prediction_mean"]
predictions

,year,district,yield_mt,log_yield,split,kfold_cv_predictions,district_yield_mean,demean_yield,prediction,district_prediction_mean,demean_prediction,demean_cv_prediction
0,2009,Chibombo,1.562466,0.408658,train,0.477336,0.499068,-0.090410,0.452742,0.494837,-0.042095,-0.017501
1,2009,Kabwe,1.899954,0.462391,train,0.480924,0.565694,-0.103302,0.483155,0.544802,-0.061647,-0.063878
2,2009,Kapiri Mposhi,1.800877,0.447294,test,NaN,0.537779,-0.090485,0.481603,0.545827,-0.064224,NaN
3,2009,Mkushi,2.678332,0.565651,train,0.635724,0.646985,-0.081334,0.619140,0.648398,-0.029258,-0.012674
4,2009,Mumbwa,1.327876,0.366960,train,0.379211,0.453685,-0.086725,0.370820,0.452709,-0.081888,-0.073498
...,...,...,...,...,...,...,...,...,...,...,...,...
931,2021,Lukulu,1.008066,0.302778,test,NaN,0.262924,0.039854,0.246322,0.247950,-0.001629,NaN
932,2021,Mongu,0.813282,0.258465,test,NaN,0.204386,0.054079,0.223268,0.229394,-0.006127,NaN
933,2021,Senanga,0.561860,0.193642,train,0.143945,0.165216,0.028426,0.140638,0.152596,-0.011958,-0.008651
934,2021,Sesheke,0.538047,0.186970,train,0.176209,0.193657,-0.006688,0.175402,0.208751,-0.033348,-0.032542


In [29]:
cv_pred = predictions[predictions.split == 'train']
r2_score(cv_pred.demean_yield, cv_pred.demean_cv_prediction)

0.3528709081517606

In [30]:
test_pred = predictions[predictions.split == 'test']
r2_score(test_pred.demean_yield, test_pred.demean_prediction)

0.4741294522739359